### Creates .json parameter files from validated catalog combinations

In [2]:
import pandas as pd
from tqdm import tqdm
import json
from cmip6_downscaling.methods.common.containers import RunParameters

In [11]:
valid_df = pd.read_csv('valid_catalog.csv')

template = {
    "bcsd": {
        "method": "",
        "obs": "ERA5",
        "model": "",
        "member": "",
        "grid_label": "gn",
        "table_id": "day",
        "scenario": "",
        "features": "",
        "variable": "",
        "train_dates": ["1981", "2010"],
        "predict_dates": "",
        "latmin": "-90",
        "latmax": "90",
        "lonmin": "-180",
        "lonmax": "180",
        "bias_correction_method": "",
        "bias_correction_kwargs": {},
        "model_type": "",
        "model_params": {},
    },
    "gard": {
        "method": "",
        "obs": "ERA5",
        "model": "",
        "member": "",
        "grid_label": "gn",
        "table_id": "day",
        "scenario": "",
        "features": "",
        "variable": "",
        "train_dates": ["1981", "2010"],
        "predict_dates": "",
        "latmin": "-90",
        "latmax": "90",
        "lonmin": "-180",
        "lonmax": "180",
        "bias_correction_method": "quantile_mapper",
        "bias_correction_kwargs": {},
        "model_type": "PureRegression",
        "model_params": {},
    },
    "maca": {
        "method": "",
        "obs": "ERA5",
        "model": "",
        "member": "",
        "grid_label": "gn",
        "table_id": "day",
        "scenario": "",
        "variable": "",
        "train_dates": ["1981", "2010"],
        "predict_dates": "",
        "latmin": "-90",
        "latmax": "90",
        "lonmin": "-180",
        "lonmax": "180",
        "year_rolling_window": 31,
        "day_rolling_window": 21,
    },
}

In [12]:
gcms = ['MIROC6', 'CanESM5', 'BCC-CSM2-MR', 'MPI-ESM1-2-HR', 'MRI-ESM2-0']
member = 'r1i1p1f1'
variables = ['tasmax', 'tasmin', 'pr']
bias_correction_kwargs = {
    "pr": {"detrend": False},
    "tasmin": {"detrend": True},
    "tasmax": {"detrend": True},
    "psl": {"detrend": False},
    "ua": {"detrend": False},
    "va": {"detrend": False},
}

In [13]:
for method in ['maca']:  # ['bcsd', 'gard', 'maca']:
    for gcm in gcms:
        for variable in variables:
            subset = valid_df[valid_df['source_id'] == gcm]

            subset = subset[subset['member_id'] == member]
            subset = subset[subset['variable_id_y'] == variable]

            for index, row in tqdm(subset.iterrows()):
                filled_template = template[method].copy()
                filled_template["method"] = method
                filled_template["member"] = row["member_id"]
                filled_template["model"] = row["source_id"]
                filled_template["variable"] = row["variable_id_y"]
                if method == 'maca' and 'hist' in row["experiment_id_y"]:
                    continue
                filled_template["scenario"] = row["experiment_id_y"]

                if method == 'maca':
                    filled_template["predict_dates"] = ["1950", "2099"]

                elif method == 'gard':
                    filled_template["features"] = [row["variable_id_y"]]
                    if 'ssp' in filled_template["scenario"]:
                        filled_template["predict_dates"] = ["2015", "2099"]
                    else:
                        filled_template["predict_dates"] = ["1950", "2014"]

                    if variable == 'pr':
                        filled_template['model_params'] = {'thresh': 0}
                        filled_template['bias_correction_kwargs'] = {"detrend": "False"}
                    elif variable in ['tasmax', 'tasmin']:
                        filled_template['bias_correction_kwargs'] = {"detrend": "True"}
                    filled_template['train_dates'] = filled_template.pop('train_dates')
                    filled_template['predict_dates'] = filled_template.pop('predict_dates')

                run_parameters = RunParameters(**filled_template)

                with open(f'{method}' + '/' + run_parameters.run_id + '.json', 'w') as f:
                    f.write(json.dumps(filled_template))

4it [00:00, 29.80it/s]
4it [00:00, 34.49it/s]
4it [00:00, 42.07it/s]
4it [00:00, 41.90it/s]
4it [00:00, 42.15it/s]
4it [00:00, 42.39it/s]
4it [00:00, 42.25it/s]
4it [00:00, 48.47it/s]
4it [00:00, 42.34it/s]
4it [00:00, 43.26it/s]
4it [00:00, 41.83it/s]
4it [00:00, 41.79it/s]
4it [00:00, 42.70it/s]
4it [00:00, 42.71it/s]
4it [00:00, 42.69it/s]


In [ ]:
obs